In [11]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [12]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import time
import shutil
from datetime import datetime, timedelta
import os
from pyspark.sql import functions as F

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,io.delta:delta-core_2.12:2.4.0")\
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


In [13]:
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType


In [14]:
# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000)\
    .load()


parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("CLOSE")
)

    # F.col("data.prices.ethereum").alias("ETH"),
    # F.col("data.prices.tether").alias("USDT"),
    # F.col("data.prices.usd-coin").alias("USDC"),
    # F.col("data.prices.ripple").alias("XRP"),
    # F.col("data.prices.cardano").alias("ADA"),
    # F.col("data.prices.dogecoin").alias("DOGE"),
    # F.col("data.prices.matic-network").alias("MATIC"),
    # F.col("data.prices.solana").alias("SOL"),
    # F.col("data.prices.litecoin").alias("LTC"),
    # F.col("data.prices.polkadot").alias("DOT"),
    # F.col("data.prices.shiba-inu").alias("SHIB"),
    # F.col("data.prices.tron").alias("TRX"),
    # F.col("data.prices.cosmos").alias("ATOM"),
    # F.col("data.prices.chainlink").alias("LINK"),
    # F.col("data.prices.stellar").alias("XLM"),
    # F.col("data.prices.near").alias("NEAR")
# )

# # Định nghĩa schema cho dữ liệu lịch sử
historical_schema = StructType([
    StructField("BASE", StringType(), True),#là mã của crypto, ví dụ BASE BTC sẽ có các row chứa dữ liệu theo các cột kế bên
    StructField("DATE", TimestampType(), True),
    StructField("OPEN", FloatType(), True),
    StructField("HIGH", FloatType(), True),
    StructField("LOW", FloatType(), True),
    StructField("CLOSE", FloatType(), True),
    StructField("VOLUME", FloatType(), True),
    StructField("YEAR", StringType(), True),
    StructField("MONTH", StringType(), True),
    StructField("__index_level_0__", StringType(), True)
])


In [ ]:
# 1. Đọc dữ liệu batch từ Parquet
historical_data_df = (
    spark.read.format("parquet")
    .load("gs://crypto-historical-data-2/ver2/BTC/2024/*")
    .select(F.col("DATE").cast("timestamp"), "CLOSE")
)

# 3. Trigger micro-batch và xử lý
def process_batch(micro_batch_df, batch_id):
    # Lọc ra giá trị gần nhất của micro-batch (dòng có timestamp mới nhất)
    micro_batch_latest_df = (
        micro_batch_df
        .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
        .filter(F.col("row_num") == 1)  # Chỉ lấy dòng có timestamp mới nhất
        .drop("row_num")  # Loại bỏ cột row_num
    )
    # Kết hợp micro-batch với dữ liệu batch (historical_data_df)
    combined_df = (
        micro_batch_latest_df
        .unionByName(historical_data_df)  # Union với dữ liệu batch
    )
    
    # Tính toán các chỉ số SMA với các cửa sổ khác nhau
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)

    combined_df = combined_df.withColumn("SMA_5", F.avg("CLOSE").over(window_spec.rowsBetween(-4, 0)))
    combined_df = combined_df.withColumn("SMA_10", F.avg("CLOSE").over(window_spec.rowsBetween(-9, 0)))
    combined_df = combined_df.withColumn("SMA_20", F.avg("CLOSE").over(window_spec.rowsBetween(-19, 0)))
    combined_df = combined_df.withColumn("SMA_50", F.avg("CLOSE").over(window_spec.rowsBetween(-49, 0)))
    combined_df = combined_df.withColumn("SMA_100", F.avg("CLOSE").over(window_spec.rowsBetween(-99, 0)))
    combined_df = combined_df.withColumn("SMA_200", F.avg("CLOSE").over(window_spec.rowsBetween(-199, 0)))

    # Sắp xếp lại theo DATE
    combined_df = combined_df.orderBy("DATE", ascending=False)

    # Ghi ra kết quả
    combined_df.write.format("console").option("truncate", False).save()
    # combined_df.write \
    # .format("csv") \
    # .option("header", "true") \
    # .option("path", "gs://indicator-crypto/sma_results/BTC") \
    # .mode("overwrite") \
    # .save()


query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()


24/12/12 22:46:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 22:46:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 22:46:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/temporary-36881924-d193-445d-882d-073ebf63c76a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/12 22:46:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/12 22:46:29 WARN AdminClientConfig: These configurations '[key.deserializer, value.

+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE               |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-12 00:00:00|101552.7|99590.94         |98948.94         |97533.79650000001|86083.29419999997|74100.0238       |68584.66789999999|
|2024-12-11 00:00:00|101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00|96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151       |68260.20674999998|
|2024-12-09 00:00:00|97399.9 |99086.0          |97991.388        |96827.963        |84170.41999999998|72860.2478       |68116.63349999998|
|2024-12-08 00:00:00|101170

24/12/12 22:46:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 22:46:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 22:46:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 22:46:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-12 22:46:53.285829|101722.0|99701.34         |99529.39         |97668.69499999999|86785.71199999997|74542.5554       |68750.90929999998|
|2024-12-12 00:00:00       |101552.7|99590.94         |98948.94         |97533.79650000001|86083.29419999997|74100.0238       |68584.66789999999|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00       |96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151     

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


...

24/12/12 23:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:35:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-12 23:35:33.840064|101894.0|99735.74         |99546.59         |97677.295        |86789.15199999997|74544.2754       |68751.76929999999|
|2024-12-12 00:00:00       |101552.7|99590.94         |98948.94         |97533.79650000001|86083.29419999997|74100.0238       |68584.66789999999|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00       |96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151     

24/12/12 23:36:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:36:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:36:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/12 23:36:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-12 23:36:04.692375|101894.0|99735.74         |99546.59         |97677.295        |86789.15199999997|74544.2754       |68751.76929999999|
|2024-12-12 00:00:00       |101552.7|99590.94         |98948.94         |97533.79650000001|86083.29419999997|74100.0238       |68584.66789999999|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00       |96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151     

24/12/12 23:36:37 ERROR MicroBatchExecution: Query [id = e1264e7b-e599-4703-ba36-852ae2a979ce, runId = aa7113c4-7b9b-409a-b35a-bb1cb0dbe0d6] terminated with error
py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/sql/utils.py", line 120, in call
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/sql/utils.py", line 117, in call
  File "/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/ipykernel_18976/2926996480.py", line 43, in process_batch
    .save()
     ^^^^^^
  File "/Users/ib